# Process VODCA v2

Downloaded from here: https://essd.copernicus.org/preprints/essd-2024-35/


## Load packages

In [ ]:
import sys
import os
import xarray as xr
import pandas as pd
from odc.geo.xr import assign_crs

## Combine files

In [ ]:
base='/g/data/os22/chad_tmp/Aus_phenology/data/VODCA_CXKu/daily_images_VODCA_CXKu/'
years = [str(i) for i in range(1988,2022)]

In [ ]:
dss=[]
for y in years:
    print(y)
    files = [base+y+'/'+i for i in os.listdir(base+y)]
    files.sort()
    ds = xr.open_mfdataset(files)
    ds = ds.rename({'lat':'latitude', 'lon':'longitude'})
    ds = assign_crs(ds['VODCA_CXKu'], crs='EPSG:4326')
    ds = ds.sel(latitude=slice(-10,-45), longitude=slice(110,155))
    ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
    dss.append(ds.compute())

vodca = xr.concat(dss, dim='time').sortby('time')

In [ ]:
vodca.mean('time').plot(robust=True)

In [ ]:
vodca.mean(['latitude','longitude']).plot(figsize=(12,4))

In [ ]:
vodca.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/VODCAv2_1988_2021.nc')